## 2. Structural Patterns

### Decorator
---

This pattern adds new features to existing objects dynamically without subclassing. The Python language have decorators as a built-in feature. Related patterns are the adapter, composite, and strategy patterns.

In [1]:
from functools import wraps


def make_blink(function):
    """Defines the decorator."""
    # This makes the decorator transparent in terms of its name and docstring
    @wraps(function)
    # Define the inner function
    def decorator():
        # Grab the return value of the function being decorated
        ret = function()
        # Add new functionality to the function being decorated
        return f'<blink>{ret}</blink>'

    return decorator


# Apply the decorator to a function
@make_blink
def hello_world():
    """Original function."""
    return 'Hello, World!'


# Check the result of decorating
print(hello_world())

# Check if the function name is still the same as that of the function being decorated
print(hello_world.__name__)

# Check if the docstring is still the same as that of the function being decorated
print(hello_world.__doc__)

<blink>Hello, World!</blink>
hello_world
Original function.


### Proxy
---

This pattern is used when creating objects that are very resource intensive by postponing their creation until absolutely necessary. A placeholder is used which will create the object when needed. In the solution, clients interact with a proxy object most of the time until the resource intensive object becomes available. The proxy object is thus responsible for the creation of the resource intensive objects. Related patterns are the adapter and decorator patterns.

In [2]:
import time


class Producer(object):
    """Define the resource intensive object."""
    def produce(self):
        print('Producer is working hard!')

    def meet(self):
        print('Producer has time to meet you now!')


class Proxy(object):
    """Define the relatively less resource intensive proxy object."""
    def __init__(self):
        self.occupied = 'No'
        self.producer = None

    def produce(self):
        """Check if producer is available."""
        print('Artist checking if producer is available...')
        
        if self.occupied == 'No':
            # If the producer is available, create a producer object
            self.producer = Producer()
            time.sleep(2)
            # Have the producer meet the guest
            self.producer.meet()
        else:
            # Otherwise, don't create a producer
            time.sleep(2)
            print('Producer is busy!')


# Instantiate a Proxy
p = Proxy()

# Have the proxy object produce until Producer is available
p.produce()

# Change the state to occupied
p.occupied = 'Yes'

# Make the Producer produce
p.produce()

Artist checking if producer is available...
Producer has time to meet you now!
Artist checking if producer is available...
Producer is busy!


### Adapter
---

The adapter pattern converts the interface of a class into one expected by the client. For example, consider the incompatible interface between a client that would like a uniform interface (`speak()`) and a server that has objects that emit Korean (`speak_korean()`) and objetcs that emit English (`speak_english()`). The solution is to use an adapter pattern that translates the method names between the client and server. Related patterns are bridges and decorators.

In [3]:
class Korean(object):
    """Korean speaker."""
    def __init__(self):
        self.name = 'Korean'

    def speak_korean(self):
        return 'An-neyong!'


class British(object):
    """English speaker."""
    def __init__(self):
        self.name = 'British'

    # Note the different method name here!
    def speak_english(self):
        return 'Hello!'


class Adapter(object):
    """This changes the generic method name to individualized method names."""
    def __init__(self, object, **adapted_method):
        """Change the name of the method."""
        self._object = object

        # Add a new dictionary item that establishes the mapping between
        # the generic method name speak() and the concrete method
        # for example, speak() could be mapped to speak_korean()
        self.__dict__.update(adapted_method)

    def __getattr__(self, attr):
        """Simply return the rest of the attributes."""
        return getattr(self._object, attr)


# List of speaker objects
objects = []

# Create a Korean object
korean = Korean()

# Create a British object
british = British()

# Append the objects to the list of speaker objects while adapting
# the generic method name to the individualized method names
objects.append(Adapter(korean, speak=korean.speak_korean))
objects.append(Adapter(british, speak=british.speak_english))

# Check that the adapter works
for obj in objects:
    print(f'{obj.name} says "{obj.speak()}"')

Korean says "An-neyong!"
British says "Hello!"


### Composite
---

The composite pattern maintains a tree data structure to represent part-whole relationships. This problem is encountered when building recursive tree data structures where elements of the tree can have their own sub-elements. An example of this is menus with multiple depths of submenus. The solution consists of three elements:

- Component  
  An abstract class.
- Child  
  A concrete class inheriting from the component class.
- Composite  
  Another concrete class that also inherits from the component class that maintains child objects by adding and removing them from a tree data structure.
  
Related patterns are the decorator, iterator, and visitor patterns.

In [4]:
class Component(object):
    """Abstract class."""
    def __init__(self, *args, **kwargs):
        pass

    def component_function(self):
        pass


class Child(Component):
    """Concrete class."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # This is where the name of the child item is stored!
        self.name = args[0]

    def component_function(self):
        # Print the name of the child item here!
        print(f'{self.name}')


class Composite(Component):
    """Concrete class that maintains the recursive tree structure."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # This is where the name of the composite object is stored!
        self.name = args[0]

        # This is where the child items are stored!
        self.children = []

    def append_child(self, child):
        """Method to add a new child item."""
        self.children.append(child)

    def remove_child(self, child):
        """Method to remove a child item."""
        self.children.remove(child)

    def component_function(self):
        # Print the name of the composite object
        print(f'{self.name}')

        # Print the names of the child items
        for child in self.children:
              child.component_function()


# Let's start building a tree of menus, starting with a submenu
sub1 = Composite('submenu_1')

# Create a new child sub-submenus
sub11 = Child('subsubmenu_11')
sub12 = Child('subsubmenu_12')

# Add the sub-submenus to the submenu
sub1.append_child(sub11)
sub1.append_child(sub12)

# Let's also build a submenu that is not a composite
sub2 = Child('submenu_2')

# Now build a top-level composite menu
top = Composite('top_menu')

# Add the composite submenu 1 and plain submenu 2
# to the list of children for the top-level menu
top.append_child(sub1)
top.append_child(sub2)

# Check that the composite pattern works
top.component_function()

top_menu
submenu_1
subsubmenu_11
subsubmenu_12
submenu_2


### Bridge
---

The bridge patterns helps with untangling unnecessarily complicated class hierarchies that manifest as unrelated parallel or orthogonal abstractions, some of which are implementation-specific and while others are implementation-independent. For example, consider an abstraction for a geometric shape where the drawing of the shape is implementation-specific while the definition of its properties and scaling are implementation-independent. The solution does not try to abstract both implementation-specific and implementation-independent classes in a single class hierarchy, but to separate the abstractions into distinct class hierarchies. Related patterns are the abstract factory and adapter patterns.

In [5]:
class DrawingAPIOne(object):
    """Implementation-specific abstraction: concrete class one."""
    def draw_circle(self, x, y, radius):
        print(f'API 1 drawing a circle at ({x}, {y}) with radius {radius}!')


class DrawingAPITwo(object):
    """Implementation-specific abstraction: concrete class two."""
    def draw_circle(self, x, y, radius):
        print(f'API 2 drawing a circle at ({x}, {y}) with radius {radius}!')


class Circle(object):
    """Implementation-independent abstraction: for example, there could be a rectangle class!"""
    def __init__(self, x, y, radius, drawing_api):
        # Initialize the necessary attributes
        self._x = x
        self._y = y
        self._radius = radius
        self._drawing_api = drawing_api

    def draw(self):
        """Implementation-specific abstraction taken care of by another class: DrawingAPI."""
        self._drawing_api.draw_circle(self._x, self._y, self._radius)

    def scale(self, percent):
        """Implementation-independent."""
        self._radius *= percent


# Build the first Circle object using API One
circle1 = Circle(1, 2, 3, DrawingAPIOne())

# Draw a circle
circle1.draw()

# Build the second Circle object using API Two
circle2 = Circle(2, 3, 4, DrawingAPITwo())

# Draw a circle
circle2.draw()

API 1 drawing a circle at (1, 2) with radius 3!
API 2 drawing a circle at (2, 3) with radius 4!
